In [2]:
!pip install tensorflow==2.17.0 keras==3.4.1 scikit-learn==1.5.2

  Obtaining dependency information for tensorflow==2.17.0 from https://files.pythonhosted.org/packages/ed/b6/62345568cd07de5d9254fcf64d7e44aacbb6abde11ea953b3cb320e58d19/tensorflow-2.17.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for keras==3.4.1 from https://files.pythonhosted.org/packages/46/43/03fa53f027e78af4a6bee3564d05cb34d9f5b924dc69c85f8ef5cb950ff1/keras-3.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for scikit-learn==1.5.2 from https://files.pythonhosted.org/packages/17/1c/ccdd103cfcc9435a18819856fbbe0c20b8fa60bfc3343580de4be13f0668/scikit_learn-1.5.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.17.0 from https://files.pythonhosted.org/packages/66/03/5c447feceb72f5a38ac2aa79d306fa5b5772f982c2b480c1329c7e382900/tensorflow_intel-2.17.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for threadpoolctl>=3.1.0 from https://files.pythonhosted.org/packages/4b/2c/ffbf

In [1]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Bidirectional, Dense, Concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [8]:
!pip install pandas
!pip install xlrd

'pip' is not recognized as an internal or external command,
operable program or batch file.


'pip' is not recognized as an internal or external command,
operable program or batch file.
'pip' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
import pandas  as pd
data = pd.read_excel(r"C:\Users\Dell\Documents\Capstone\dict.xlsx")


sanskrit_words = data['Sanskrit'].astype(str)
english_words = data['English'].astype(str)

# Hyperparameters
embedding_dim = 128
units = 256
max_length_sanskrit = 2  # Word-level data, so length = 1
max_length_english = 2  # Word-level data, so length = 1

# Tokenizer for Sanskrit
tokenizer_sanskrit = Tokenizer(filters='', lower=False)
tokenizer_sanskrit.fit_on_texts(sanskrit_words)
sanskrit_sequences = tokenizer_sanskrit.texts_to_sequences(sanskrit_words)
sanskrit_padded = pad_sequences(sanskrit_sequences, maxlen=max_length_sanskrit, padding='post')

# Tokenizer for English
tokenizer_english = Tokenizer(filters='', lower=False)
tokenizer_english.fit_on_texts(english_words)
english_sequences = tokenizer_english.texts_to_sequences(english_words)
english_padded = pad_sequences(english_sequences, maxlen=max_length_english, padding='post')

In [4]:
# Vocabulary sizes
vocab_size_sanskrit = len(tokenizer_sanskrit.word_index) + 1
vocab_size_english = len(tokenizer_english.word_index) + 1

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(sanskrit_padded, english_padded, test_size=0.2, random_state=42)

In [7]:
# Encoder (Sanskrit)
encoder_inputs = Input(shape=(max_length_sanskrit,))
encoder_embedding = Embedding(input_dim=vocab_size_sanskrit, output_dim=embedding_dim)(encoder_inputs)
encoder_biLSTM = Bidirectional(LSTM(units, return_sequences=False, return_state=True))

# Forward and backward states
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_biLSTM(encoder_embedding)
state_h = Concatenate()([forward_h, backward_h])  # Concatenate forward and backward hidden states
state_c = Concatenate()([forward_c, backward_c])  # Concatenate forward and backward cell states

# Decoder (English)
decoder_inputs = Input(shape=(max_length_english,))
decoder_embedding = Embedding(input_dim=vocab_size_english, output_dim=embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(units * 2, return_sequences=True, return_state=True)

# Using the encoder's states as initial states for the decoder
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])
decoder_dense = Dense(vocab_size_english, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

TypeError: Exception encountered when calling Bidirectional.call().

[1mcan only concatenate list (not "tuple") to list[0m

Arguments received by Bidirectional.call():
  • args=('<KerasTensor shape=(None, 2, 128), dtype=float32, sparse=False, name=keras_tensor_5>',)
  • kwargs={'mask': 'None'}

In [5]:
# Encoder (Sanskrit)
encoder_inputs = Input(shape=(max_length_sanskrit,))
encoder_embedding = Embedding(input_dim=vocab_size_sanskrit, output_dim=embedding_dim)(encoder_inputs)
encoder_biLSTM = Bidirectional(LSTM(units, return_sequences=False, return_state=True))

# Forward and backward states
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_biLSTM(encoder_embedding)
state_h = Concatenate()([forward_h, backward_h])  # Concatenate forward and backward hidden states
state_c = Concatenate()([forward_c, backward_c])  # Concatenate forward and backward cell states

# Decoder (English)
decoder_inputs = Input(shape=(max_length_english,))
decoder_embedding = Embedding(input_dim=vocab_size_english, output_dim=embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(units * 2, return_sequences=True, return_state=True)

# Using the encoder's states as initial states for the decoder
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])
decoder_dense = Dense(vocab_size_english, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Full model combining encoder and decoder
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# Training

# Shift the target English words by one for teacher forcing
y_train_shifted = np.expand_dims(np.roll(y_train, -1, axis=1), -1)
y_test_shifted = np.expand_dims(np.roll(y_test, -1, axis=1), -1)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 2)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_2 (Embedding)   │ (None, 2, 128)         │        161,920 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_3             │ (None, 2)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_1           │ [(None, 512), (None,   │        788,480 │ embedding_2[0][0]      │
│ (Bidirectional)           │ 256), (None, 256),     │                │                        │
│                           │ (None, 256), (None,    │                │                        │
│                           │ 256)]                  │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_3 (Embedding)   │ (None, 2, 128)         │        150,400 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_2             │ (None, 512)            │              0 │ bidirectional_1[0][1], │
│ (Concatenate)             │                        │                │ bidirectional_1[0][3]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_3             │ (None, 512)            │              0 │ bidirectional_1[0][2], │
│ (Concatenate)             │                        │                │ bidirectional_1[0][4]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_3 (LSTM)             │ [(None, 2, 512),       │      1,312,768 │ embedding_3[0][0],     │
│                           │ (None, 512), (None,    │                │ concatenate_2[0][0],   │
│                           │ 512)]                  │                │ concatenate_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 2, 1175)        │        602,775 │ lstm_3[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,016,343 (11.51 MB)

 Trainable params: 3,016,343 (11.51 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Train the model
history = model.fit(
    [X_train, y_train],
    y_train_shifted,
    epochs=30,
    batch_size=32,
    validation_data=([X_test, y_test], y_test_shifted)
)

Epoch 1/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.6557 - loss: 1.4070 - val_accuracy: 0.5342 - val_loss: 5.2317
Epoch 2/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6460 - loss: 1.3972 - val_accuracy: 0.5326 - val_loss: 5.3199
Epoch 3/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6510 - loss: 1.3792 - val_accuracy: 0.5311 - val_loss: 5.2437
Epoch 4/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6726 - loss: 1.3470 - val_accuracy: 0.5435 - val_loss: 5.3398
Epoch 5/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6754 - loss: 1.3259 - val_accuracy: 0.5326 - val_loss: 5.3407
Epoch 6/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6839 - loss: 1.2644 - val_accuracy: 0.5373 - val_loss: 5.2917
Epoch 7/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6619 - loss: 1.2894 - val_accuracy: 0.5404 - val_loss: 5.3076
Epoch 8/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6751 - loss: 1.2832 - val_accuracy: 0.5388 - v

In [16]:
# Evaluation
# Predict on the test data
predictions = model.predict([X_test, y_test])

# Convert predictions back to words
predicted_words = np.argmax(predictions, axis=-1)

# Convert indices back to words
y_test_flat = y_test.flatten()
predicted_flat = predicted_words.flatten()

# Inverse map from index to word
index_to_word_english = {index: word for word, index in tokenizer_english.word_index.items()}

# Convert predictions to words
y_test_words = [index_to_word_english.get(index, '') for index in y_test_flat]
predicted_words = [index_to_word_english.get(index, '') for index in predicted_flat]

# Calculate accuracy
accuracy = accuracy_score(y_test_flat, predicted_flat)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Display some test cases with their predictions
for i in range(50):
    print(f"Input (Sanskrit): {sanskrit_words[i]}")
    print(f"Translation(English): {predicted_words[i]}\n")


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Accuracy: 0.00%
Input (Sanskrit): अहम्
Translation(English): 

Input (Sanskrit): माम्
Translation(English): decide

Input (Sanskrit): त्वम्
Translation(English): 

Input (Sanskrit): गच्छ
Translation(English): thunder

Input (Sanskrit): अगच्छत्
Translation(English): 

Input (Sanskrit): सः
Translation(English): decide

Input (Sanskrit): सा
Translation(English): 

Input (Sanskrit): तत्
Translation(English): firing

Input (Sanskrit): वयम्
Translation(English): 

Input (Sanskrit): ते
Translation(English): design

Input (Sanskrit): एतत्
Translation(English): 

Input (Sanskrit): तत्
Translation(English): design

Input (Sanskrit): कः
Translation(English): 

Input (Sanskrit): किम्
Translation(English): bring

Input (Sanskrit): कुत्र
Translation(English): 

Input (Sanskrit): कदा
Translation(English): stir

Input (Sanskrit): किमर्थम्
Translation(English): 

Input (Sanskrit): कथम्
Translation(English): firing

Input (Sanskrit): कतमः
Translation(English): 

I

In [18]:
model_word = model.save("model_word.h5")



In [27]:
from keras.models import load_model
loaded_model = load_model('model_word.h5')
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(
    [X_train, y_train],
    y_train_shifted,
    epochs=5,
    batch_size=32,
    validation_data=([X_test, y_test], y_test_shifted)
)

Epoch 1/5
41/41 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.7794 - loss: 0.8846 - val_accuracy: 0.5652 - val_loss: 5.3047
Epoch 2/5
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7830 - loss: 0.8684 - val_accuracy: 0.5714 - val_loss: 5.4240
Epoch 3/5
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8198 - loss: 0.8176 - val_accuracy: 0.5792 - val_loss: 5.4367
Epoch 4/5
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8232 - loss: 0.7789 - val_accuracy: 0.5807 - val_loss: 5.5125
Epoch 5/5
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8306 - loss: 0.7559 - val_accuracy: 0.5776 - val_loss: 5.5303


In [33]:
# import numpy as np
# import keras
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.optimizers import Adam

# class HumanFeedbackAgent:
#     def __init__(self, model):
#         self.model = model  # Pre-trained model
#         self.learning_rate = 0.001
#         self.discount_factor = 0.99  # Future reward discount
#         self.optimizer = Adam(learning_rate=self.learning_rate)

#     # Function to calculate rewards based on human feedback
#     def get_reward(self, human_feedback):
#         if human_feedback == 'positive':
#             return 1.0
#         elif human_feedback == 'negative':
#             return -1.0
#         else:
#             return 0.0

#     # Update policy based on feedback
#     def train(self, state, action, reward):
#         with tf.GradientTape() as tape:
#             # Forward pass through the model to get predictions
#             prediction = self.model(state)

#             # Loss function (we'll use policy gradient with rewards)
#             loss = -reward * tf.math.log(prediction[action])

#         # Compute gradients and update the model weights
#         gradients = tape.gradient(loss, self.model.trainable_variables)
#         self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))

#     # Predict the best action (e.g., translation output)
#     def predict(self, state):
#         action_probs = self.model(state)
#         return np.argmax(action_probs, axis=1)

# # Define the model architecture for RL agent
# def create_model(input_dim, output_dim):
#     model = Sequential()
#     model.add(Dense(128, input_dim=input_dim, activation='relu'))
#     model.add(Dense(64, activation='relu'))
#     model.add(Dense(output_dim, activation='softmax'))  # Output probabilities for actions
#     return model

# # Initialize the model
# input_dim = ...  # Dimensionality of the input
# output_dim = ...  # Number of possible actions (e.g., vocabulary size)
# pretrained_model = create_model(input_dim, output_dim)

# # Initialize the agent with the model
# agent = HumanFeedbackAgent(pretrained_model)

# # Example state (input data for prediction)
# state = np.array([...])  # Input data (e.g., tokenized text)

# # Agent predicts output
# predicted_action = agent.predict(state)

# # Simulate human feedback (in practice, you'd collect this from a user)
# human_feedback = 'positive'  # Can be 'positive' or 'negative'

# # Convert feedback to a reward
# reward = agent.get_reward(human_feedback)

# # Train agent based on feedback
# agent.train(state, predicted_action, reward)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Invalid dtype: ellipsis

In [36]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import tensorflow as tf

input_dim = 100 # Replace with the actual dimensionality of your input
output_dim = 1000 # Replace with the actual number of possible actions (e.g., vocabulary size)
pretrained_model = create_model(input_dim, output_dim)

class HumanFeedbackAgent:
    def __init__(self, model):
        self.model = model  # Pre-trained model
        self.learning_rate = 0.001
        self.discount_factor = 0.99  # Future reward discount
        self.optimizer = Adam(learning_rate=self.learning_rate)

    # Function to calculate rewards based on human feedback
    def get_reward(self, human_feedback):
        if human_feedback == 'positive':
            return 1.0
        elif human_feedback == 'negative':
            return -1.0
        else:
            return 0.0

    # Update policy based on feedback
    def train(self, state, action, reward):
        with tf.GradientTape() as tape:
            # Forward pass through the model to get predictions
            prediction = self.model(state)

            # Loss function (we'll use policy gradient with rewards)
            loss = -reward * tf.math.log(prediction[action])

        # Compute gradients and update the model weights
        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))

    # Predict the best action (e.g., translation output)
    def predict(self, state):
        action_probs = self.model(state)
        return np.argmax(action_probs, axis=1)

# Define the model architecture for RL agent
def create_model(input_dim, output_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(output_dim, activation='softmax'))  # Output probabilities for actions
    return model

# Initialize the model
input_dim = ...  # Dimensionality of the input
output_dim = ...  # Number of possible actions (e.g., vocabulary size)
pretrained_model = create_model(input_dim, output_dim)

# Initialize the agent with the model
agent = HumanFeedbackAgent(pretrained_model)

# Example state (input data for prediction)
state = np.array([...])  # Input data (e.g., tokenized text)

# Agent predicts output
predicted_action = agent.predict(state)

# Simulate human feedback (in practice, you'd collect this from a user)
human_feedback = 'positive'  # Can be 'positive' or 'negative'

# Convert feedback to a reward
reward = agent.get_reward(human_feedback)

# Train agent based on feedback
agent.train(state, predicted_action, reward)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Invalid dtype: ellipsis